### Recommender System ###

In [3]:
#  Imports

import random   # To generate random transactions
import pandas as pd

# Import tools for the Apriori algorithm and one-hot encoding

from mlxtend.preprocessing import TransactionEncoder # To convert transactions to a matrix
from mlxtend.frequent_patterns import apriori, association_rules  # To find frequent item and 
#To generate association rules

import pymysql  

In [13]:
#  – Load Jumia products from database

def get_connection():
    """
    Open a MySQL connection.
    Change user/password/database if needed.
    """
    conn = pymysql.connect(
        host="localhost",
        user="root",
        password="placali200",
        database="Project_DM",
        charset="utf8mb4",
        autocommit=True
    )
    return conn


# Load the Jumia table
conn = get_connection()
jumia_df = pd.read_sql("SELECT * FROM jumia_products", conn)

# Convert brand_name to string and remove extra spaces 
jumia_df["item_name"] = jumia_df["brand_name"].astype(str).str.strip()

# Remove empty names if any
jumia_df = jumia_df[jumia_df["item_name"] != ""]

print("Number of Jumia products:", len(jumia_df))
jumia_df[["jumia_product_id", "item_name"]].head()


Number of Jumia products: 1232


C:\Users\PC\AppData\Local\Temp\ipykernel_21468\2279243231.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  jumia_df = pd.read_sql("SELECT * FROM jumia_products", conn)


,jumia_product_id,item_name
0,1,GUEETON Clé USB OTG en métal 128 Go étanche av...
1,2,GUEETON GUEETON Souris Sans Fil Rechargeable 2...
2,3,GUEETON 128 GB Metal Usb Flash Drive Adapter C...
3,4,GUEETON Souris Sans Fil Rechargeable 2.4G
4,5,GUEETON Clé USB - OTG 128 Go Micro USB Type C ...


In [9]:
#  – List of product names (Jumia)

# Create a simple Python list with all product names
jumia_items = jumia_df["item_name"].tolist()

print("Example product names:")
for name in jumia_items[:5]:
    print("*", name)


Example product names:
* GUEETON Clé USB OTG en métal 128 Go étanche avec embossage 3D
* GUEETON GUEETON Souris Sans Fil Rechargeable 2.4G Bluetooth Dual Mode
* GUEETON 128 GB Metal Usb Flash Drive Adapter Cable 2 In 1 Set
* GUEETON Souris Sans Fil Rechargeable 2.4G
* GUEETON Clé USB - OTG 128 Go Micro USB Type C 3 in 1


In [17]:
#Each list inside jumia_transactions represents one ‘customer basket’.
#We simulate 500 baskets to be able to apply Apriori and learn which products appear together.”

#  – Simulate transactions (Jumia)

def simulate_transactions(item_names, n_transactions=500,
                          min_items=1, max_items=4):
    """
    Simulate n_transactions shopping baskets.

    Each transaction is a list of product names.
    n_transactions: number of transactions to simulate(number of basket)
    min_items: minimum number of products in one transaction
    max_items: maximum number of products in one transaction
    """
    transactions = []  # List that will store all transactions
    n_items_total = len(item_names) # Total number of different products

    for _ in range(n_transactions):
        # Choose a random basket size between min_items and max_items
        size = random.randint(min_items, max_items)
         # This cannot choose more items than we have in total
        size = min(size, n_items_total)  

        
        # Randomly choose 'size' DIFFERENT products for this basket
        # Randomly pick 'basket' different items from the list
        basket = random.sample(item_names, size)
        
         # Add this basket to the list of transactions
        transactions.append(basket)

    return transactions


# created 500 simulated transactions for Jumia products
jumia_transactions = simulate_transactions(
    jumia_items,
    n_transactions=500,
    min_items=1,
    max_items=4
)

print("Example Jumia transactions (names):")
for t in jumia_transactions[:5]:
    print("***", t)


Example Jumia transactions (names):
*** ['Lumineuse Souris Sans Fil - 2.4G - Noir-DR-SHUBIAO-03', 'GUEETON Métal Support Pour Ordinateur Portable En', "Epson Imprimante Couleur EcoTank L3150 - 3250 - Grande Capacité D'Impr...", 'Combo HUB USB 2.0 - 3 Ports USB - Lecteur De Carte SD / TF Lecteur De ...']
*** ['RichRipple Souris filaire à LED colorées - Souris bureautique ergonomi...']
*** ['Wolf Ensemble Clavier + Souris Clavier Souris Avec Lumière RGB']
*** ['Lenvii Etiquette Thermique - 50*25*250PCS 4-proof - 8R - Blanc', 'Lenovo Chargeur LENO Type-C / 65Watt', "Kit Recharge D'Encre Compatible Canon G2400/2410/3400/3410 - 490 - Noi...", 'Adaptateur Multiports USB HUB 8 Ports 2.0 - Noir']
*** ['Support pour ordinateurs portables réglable - Noir', "Kit Recharge D'Encre 4 Couleurs - Noir/Rouge/Jaune/Bleu 490", 'Jabra Casque filaire optimisé 7599-823-309 UC Voice 750 - Call center', 'Hp Souris Optique Filaire USB X500']


In [21]:
#  – One-hot encoding (Jumia):
# We convert the list of baskets into a matrix for Apriori:
# - one row = one transaction
# - one column = one product


# Create the TransactionEncoder object
# TransactionEncoder converts list-of-lists into a binary matrix
te_jumia = TransactionEncoder()

# fit() learns all unique items, transform() creates the matrix True/False
# Learn the items and transform the list of transactions into a boolean matrix
jumia_array = te_jumia.fit(jumia_transactions).transform(jumia_transactions)

# Convert the matrix to a DataFrame with item names as columns
jumia_ohe = pd.DataFrame(jumia_array, columns=te_jumia.columns_)


# Convert boolean values (True/False) to integers (1/0)
jumia_ohe = jumia_ohe.astype(int)

print("One-hot matrix shape (rows=transactions, cols=products):", jumia_ohe.shape)
jumia_ohe.head()


One-hot matrix shape (rows=transactions, cols=products): (500, 737)


,1 X Seringues De Pate Thermique 30g CPU GPU Processeur Dissipateur - G...,10 Inch LCD Writing Tablet Electronic Digital Drawing Board,10’‘Sans Fil Bluetooth Clavier Applicable Téléphone Tablette,150mbps USB Wifi Adaptateur Carte Réseau Sans Fil Usb Ethernet Adaptat...,17A Toner - Laserjet - CF217A - 1600 Pages - Noir,2 Bouteilles D'encre Noir Compatible Canon GI-490 G1400 G1800 G2400 G2...,2 Coupleur Adaptateur Connecteur LAN F/F Câble Réseau Ethernet RJ45,2 PCS Manette De Jeu Contrôleur Pour PC Ordinateur Portable,5 Piles Cemos Lithium 3v 1616,8.5 Inch LCD Writing Tablet Electronic Digital Drawing Board,...,Ventilateur Portable Pliable - 5 Vitesses - Rotatif à 90° - Rechargeab...,Vidéo Projecteur A10,Vimax Carte Mémoire TF Micro SD 128GB,WSD P10 ordinateur portable Intel N5095-Écran 15.6'' 16 Go+512 Go ordi...,Webcam USB 2K 1080P transparente avec microphones pour visioconférence...,Wolf Ensemble Clavier + Souris Clavier Souris Avec Lumière RGB,Xiaomi MI Répéteur (amplificateur) Wifi N300 300Mbps - Blanc,Xiaomi MI Répéteur (amplificateur) Wifi Pro 300Mbps - Noir,Xraydisk Disque SSD --- 512gb,Xraydisk Xraydisk SSD --- 512 Go
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
#  – Apriori (frequent itemsets) for Jumia

MIN_SUPPORT = 0.002   # 0.2%(appears at least 1 time) of all transactions


# Apply Apriori to find frequent itemsets in Jumia transactions
jumia_itemsets = apriori(
    jumia_ohe,        ## One-hot encoded Jumia transactions
    min_support=MIN_SUPPORT,
    use_colnames=True   #Use item names instead of column indices
)

print("Number of frequent itemsets:", len(jumia_itemsets))
jumia_itemsets.head()


C:\Users\PC\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Number of frequent itemsets: 2791


,support,itemsets
0,0.002,(1 X Seringues De Pate Thermique 30g CPU GPU P...
1,0.002,(10 Inch LCD Writing Tablet Electronic Digital...
2,0.002,(10’‘Sans Fil Bluetooth Clavier Applicable Tél...
3,0.002,(150mbps USB Wifi Adaptateur Carte Réseau Sans...
4,0.006,(17A Toner - Laserjet - CF217A - 1600 Pages - ...


In [27]:
#  – Association rules for Jumia

MIN_CONFIDENCE = 0.1   #soit 10%


# Generate association rules from the frequent itemsets
jumia_rules = association_rules(
    jumia_itemsets,
    metric="confidence",  # We use confidence as the main metric
    min_threshold=MIN_CONFIDENCE
)

# Sort the rules by lift (higher lift = stronger relationship)
jumia_rules = jumia_rules.sort_values(by="lift", ascending=False)

print("Number of rules:", len(jumia_rules))
jumia_rules[["antecedents", "consequents", "support", "confidence", "lift"]].head()


Number of rules: 8267


,antecedents,consequents,support,confidence,lift
6049,(Lot De 2 Cartouches Compatible Toners 05A - N...,(TOPLINK Câble Réseaux FTP Cat6 - 25m- 25 Mètr...,0.002,1.0,500.0
2355,(Pince à Sertir Professionnelle - RJ45),(Mini Ventilateur portable Électrique portatif...,0.002,1.0,500.0
5530,"(Câble D'imprimante USB 2.0 - Noir, KIT DE REC...",(Logitec Logitech Casque Filaire H340 Avec Mic...,0.002,1.0,500.0
5527,(Câble D'alimentation Pour Chargeur De PC Port...,"(KingSsd Disque Ssd 512Gb Haute Vitesse, VIC ♛...",0.002,1.0,500.0
5522,"(KingSsd Disque Ssd 512Gb Haute Vitesse, VIC ♛...",(Câble D'alimentation Pour Chargeur De PC Port...,0.002,1.0,500.0


In [30]:
# Cell 8 – Helper function to print rules in a human way (Jumia)


# Function to print association rules in a readable format
def print_readable_rules(rules_df, top_n=10):
    """
    Print the top_n rules in a readable format:
        rules_df= DataFrame with association rules
        top_n= number of rules to print
       A person who bought: X
       also bought:        Y

    support    = how often X and Y appear together (fraction of all transactions)
    confidence = how often Y appears when X appears (P(Y | X))
    lift       = how many times the co-occurrence of X and Y is stronger
                 than random independence.    
    """

        # If there are no rules, just print a message
    if rules_df.empty:
        print("No rules to display.")
        return


    # Loop over the first top_n rows
    for _, row in rules_df.head(top_n).iterrows():
        antecedent = ", ".join(list(row["antecedents"]))
        consequent = ", ".join(list(row["consequents"]))

        print(f"A person who bought: {antecedent}")
        print(f"also bought:        {consequent}")
        print(
            f"support={row['support']:.3f}, "
            f"confidence={row['confidence']:.3f}, "
            f"lift={row['lift']:.3f}"
        )
        print("-" * 50)


print("=== Example Jumia rules ===")
print_readable_rules(jumia_rules, top_n=5)
 

=== Example Jumia rules ===
A person who bought: Lot De 2 Cartouches Compatible Toners 05A - Noir
also bought:        TOPLINK Câble Réseaux FTP Cat6 - 25m- 25 Mètres - RJ-45 - Gris, Hp Cartouche d'encre 938 Bleu pour 9730 - 9720
support=0.002, confidence=1.000, lift=500.000
--------------------------------------------------
A person who bought: Pince à Sertir Professionnelle - RJ45
also bought:        Mini Ventilateur portable Électrique portatif rechargeable
support=0.002, confidence=1.000, lift=500.000
--------------------------------------------------
A person who bought: Câble D'imprimante USB 2.0 - Noir, KIT DE RECHARGE D'ENCRE POUR IMPRIMANTE COMPATIBLE CANON PIXMA G2420 /...
also bought:        Logitec Logitech Casque Filaire H340 Avec Micro USB
support=0.002, confidence=1.000, lift=500.000
--------------------------------------------------
A person who bought: Câble D'alimentation Pour Chargeur De PC Portable
also bought:        KingSsd Disque Ssd 512Gb Haute Vitesse, VIC ♛128G

In [32]:
print("=== Example Jumia rules ===")
print_readable_rules(jumia_rules, top_n=1)


=== Example Jumia rules ===
A person who bought: Lot De 2 Cartouches Compatible Toners 05A - Noir
also bought:        TOPLINK Câble Réseaux FTP Cat6 - 25m- 25 Mètres - RJ-45 - Gris, Hp Cartouche d'encre 938 Bleu pour 9730 - 9720
support=0.002, confidence=1.000, lift=500.000
--------------------------------------------------


### Coin Afrique ### 

In [74]:
# – Imports

import random
import pandas as pd

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

import pymysql 

In [76]:
#  – Load CoinAfrique cars from database

conn = get_connection()  

coin_df = pd.read_sql("SELECT * FROM coin_afrique_cars", conn)

# Build a clean product name brand Model
coin_df["item_name"] = (
    coin_df["brand"].astype(str).str.strip()
    + " "
    + coin_df["model"].astype(str).str.strip()
).str.strip()

# Remove rows where item_name is empty
coin_df = coin_df[coin_df["item_name"] != ""]

print("Number of CoinAfrique cars:", len(coin_df))
coin_df[["coin_afrique_id", "item_name"]].head()


Number of CoinAfrique cars: 1460


C:\Users\PC\AppData\Local\Temp\ipykernel_12832\3559022364.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  coin_df = pd.read_sql("SELECT * FROM coin_afrique_cars", conn)


,coin_afrique_id,item_name
0,1233,Volkswagen passat 2013
1,1234,Toyota Camry 2022
2,1235,Suzuki grand Vitara 2023
3,1236,Kia Forte 2020
4,1237,Mitsubishi pajero 2018


In [78]:
# – List of car names

coin_items = coin_df["item_name"].tolist()

print("Example CoinAfrique car names:")
for name in coin_items[:5]:
    print("•", name)


Example CoinAfrique car names:
• Volkswagen passat 2013
• Toyota Camry 2022
• Suzuki grand Vitara 2023
• Kia Forte 2020
• Mitsubishi pajero 2018


In [80]:
# 
def simulate_transactions(item_names, n_transactions=500,
                          min_items=1, max_items=4):
    """
    Simulate n_transactions shopping baskets.

    Each transaction is a list of product names.
    
    """
    transactions = []
    n_items_total = len(item_names)

    for _ in range(n_transactions):
        # random basket size
        size = random.randint(min_items, max_items)
        size = min(size, n_items_total)  # safety

        
        basket = random.sample(item_names, size)

        transactions.append(basket)

    return transactions


In [82]:

#  – Simulate CoinAfrique transactions (brand-based)

# Group cars by brand: brand -> list of item_name
brand_groups = (
    coin_df
    .groupby("brand")["item_name"]
    .apply(list)
    .to_dict()
)

def simulate_coin_transactions_brand_based(n_transactions=500,
                                           min_items=1,
                                           max_items=3):
    """
    Simulate n_transactions baskets for CoinAfrique.

    Most of the time, a basket will contain several cars
    from the same brand. This creates repeated combinations
    so that Apriori can find meaningful rules like:
       "A person who bought Toyota Corolla also bought Toyota Yaris".
    """
    transactions = []

    all_items = coin_items  # all car names
    n_items_total = len(all_items)

    for _ in range(n_transactions):
        basket = []

        r = random.random()

        # 80% of the time: pick cars from the same brand
        if r < 0.8 and brand_groups:
            brand = random.choice(list(brand_groups.keys()))
            models = brand_groups[brand]

            # number of cars in basket
            size = random.randint(min_items, max_items)
            size = min(size, len(models))

            # choose different models from that brand
            basket = random.sample(models, size)

        # 20% of the time: random mix of any cars
        else:
            size = random.randint(min_items, max_items)
            size = min(size, n_items_total)
            basket = random.sample(all_items, size)

        transactions.append(basket)

    return transactions


coin_transactions = simulate_coin_transactions_brand_based(
    n_transactions=500,
    min_items=1,
    max_items=3
)

print("Example CoinAfrique transactions (names):")
for t in coin_transactions[:5]:
    print("•", t)


Example CoinAfrique transactions (names):
• ['Chevrolet aveo 2006', 'Chevrolet trailblazer 2022', 'Chevrolet Tahoe 2022']
• ['CITROEN C5 2022']
• ['BMW X3 2017', 'Hyundai Tucson 2022']
• ['Toyota Corolla 2002 gro moteur', 'Suzuki dzire 2025']
• ['Bmw X5 M X-Drive  2017']


In [84]:
#  – One-hot encoding (CoinAfrique)

te_coin = TransactionEncoder()

coin_array = te_coin.fit(coin_transactions).transform(coin_transactions)

coin_ohe = pd.DataFrame(coin_array, columns=te_coin.columns_)

# 
coin_ohe = coin_ohe.astype(int)

print("One-hot matrix (CoinAfrique) shape:", coin_ohe.shape)
coin_ohe.head()



One-hot matrix (CoinAfrique) shape: (500, 397)


,4x4 Mitsubishi pajero,4x4 pick-up Toyota hilux,AC Changan Cs15 2020,AUDI Q5 2009,Abarth OT 2025,Abarth ot 2025,Acura integra 2025,Audi A4 2023,Audi A5 2017,Audi A6 2019,...,Volkswagen passat 2013,Volkswagen polo 1999,Whey Mocha 2025,Yamaha XC 2015,kia Sorento 2014,kia sorento 2020,kia sportage 2012,kia sportage 2020,mercedes-benz gls580 2021,peugeot 308 2008
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
#  – Apriori (frequent itemsets) for CoinAfrique

MIN_SUPPORT_COIN = 0.003   # 0.3% of all transactions (~1.5 transactions out of 500)

coin_itemsets = apriori(
    coin_ohe,
    min_support=MIN_SUPPORT_COIN,
    use_colnames=True
)

print("Number of frequent itemsets (CoinAfrique):", len(coin_itemsets))
coin_itemsets.head()


Number of frequent itemsets (CoinAfrique): 188


C:\Users\PC\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.012,(4x4 Mitsubishi pajero)
1,0.008,(4x4 pick-up Toyota hilux)
2,0.006,(AC Changan Cs15 2020)
3,0.004,(AUDI Q5 2009)
4,0.014,(Abarth OT 2025)


In [88]:
#  – Association rules for CoinAfrique

MIN_CONFIDENCE_COIN = 0.1   # 10%

coin_rules = association_rules(
    coin_itemsets,
    metric="confidence",
    min_threshold=MIN_CONFIDENCE_COIN
)

coin_rules = coin_rules.sort_values(by="lift", ascending=False)

print("Number of CoinAfrique rules:", len(coin_rules))
coin_rules[["antecedents", "consequents", "support", "confidence", "lift"]].head()


Number of CoinAfrique rules: 60


,antecedents,consequents,support,confidence,lift
59,(Jetour Changan 2025),"(Jetour Haval H2 2020, Jetour Julion 2022)",0.004,1.000000,166.666667
22,(Haval H9 2019),(Haval h6 gt 2025),0.004,1.000000,166.666667
54,"(Jetour Haval H2 2020, Jetour Julion 2022)",(Jetour Changan 2025),0.004,0.666667,166.666667
53,(kia sportage 2020),(kia sportage 2012),0.004,0.666667,166.666667
52,(kia sportage 2012),(kia sportage 2020),0.004,1.000000,166.666667


In [90]:
# 

def print_readable_rules(rules_df, top_n=10):
    """
    Print the top_n rules in a readable format:

       A person who bought: X
       also bought:        Y

    support    = how often X and Y appear together (fraction of all transactions)
    confidence = how often Y appears when X appears (P(Y | X))
    lift       = how many times the co-occurrence of X and Y is stronger
                 than random independence.
    """
    if rules_df.empty:
        print("No rules to display.")
        return

    for _, row in rules_df.head(top_n).iterrows():
        antecedent = ", ".join(list(row["antecedents"]))
        consequent = ", ".join(list(row["consequents"]))

        print(f"A person who bought: {antecedent}")
        print(f"also bought:        {consequent}")
        print(
            f"support={row['support']:.3f}, "
            f"confidence={row['confidence']:.3f}, "
            f"lift={row['lift']:.3f}"
        )
        print("-" * 70)


In [92]:
print("=== Example CoinAfrique rules ===")
print_readable_rules(coin_rules, top_n=5)


=== Example CoinAfrique rules ===
A person who bought: Jetour Changan 2025
also bought:        Jetour Haval H2 2020, Jetour Julion  2022
support=0.004, confidence=1.000, lift=166.667
----------------------------------------------------------------------
A person who bought: Haval H9 2019
also bought:        Haval h6 gt 2025
support=0.004, confidence=1.000, lift=166.667
----------------------------------------------------------------------
A person who bought: Jetour Haval H2 2020, Jetour Julion  2022
also bought:        Jetour Changan 2025
support=0.004, confidence=0.667, lift=166.667
----------------------------------------------------------------------
A person who bought: kia sportage 2020
also bought:        kia sportage 2012
support=0.004, confidence=0.667, lift=166.667
----------------------------------------------------------------------
A person who bought: kia sportage 2012
also bought:        kia sportage 2020
support=0.004, confidence=1.000, lift=166.667
--------------------

In [94]:
#  – Recommendation for one car



# 

def recommend_for_item(item_name, rules_df,
                       top_k=3, min_confidence=0.1):
    """
    Recommend up to top_k products for a given item_name,
    based on the association rules.
    """
    recommendations = []

    for _, row in rules_df.iterrows():
        # check if item_name is part of the antecedent of the rule
        if item_name in row["antecedents"] and row["confidence"] >= min_confidence:
            for c in row["consequents"]:
                if c != item_name and c not in recommendations:
                    recommendations.append(c)
                if len(recommendations) >= top_k:
                    break
        if len(recommendations) >= top_k:
            break

    return recommendations

In [96]:
# – Example recommendation for a car

example_car = random.choice(coin_items)
print("Clicked car:", example_car)
print("Recommended cars:")
for car in recommend_for_item(example_car, coin_rules, top_k=3):
    print(" →", car)


Clicked car: Ssangyong Rexton 2018
Recommended cars:
 → Ssangyong Tivoli  2016


In [ ]:
##

In [100]:
#  – Build mappings between names and IDs

# For Jumia
jumia_name_to_id = dict(zip(jumia_df["item_name"], jumia_df["jumia_product_id"]))
jumia_id_to_name = dict(zip(jumia_df["jumia_product_id"], jumia_df["item_name"]))

# For CoinAfrique
coin_name_to_id = dict(zip(coin_df["item_name"], coin_df["coin_afrique_id"]))
coin_id_to_name = dict(zip(coin_df["coin_afrique_id"], coin_df["item_name"]))


In [102]:
#  – Build Jumia rules with IDs

jumia_rules_rows = []

for _, row in jumia_rules.iterrows():
    antecedents = list(row["antecedents"])
    consequents = list(row["consequents"])

    for left in antecedents:
        for right in consequents:
            if left == right:
                continue

            left_id = jumia_name_to_id.get(left)
            right_id = jumia_name_to_id.get(right)

            if left_id is None or right_id is None:
                continue

            jumia_rules_rows.append({
                "left_product_id": left_id,
                "left_product_name": left,
                "right_product_id": right_id,
                "right_product_name": right,
                "support": row["support"],
                "confidence": row["confidence"],
                "lift": row["lift"]
            })

jumia_rules_id_df = pd.DataFrame(jumia_rules_rows)
jumia_rules_id_df = jumia_rules_id_df[jumia_rules_id_df["confidence"] >= 0.1] 

jumia_rules_id_df.head()


,left_product_id,left_product_name,right_product_id,right_product_name,support,confidence,lift
0,392,1 X Seringues De Pate Thermique CPU GPU Proces...,801,Adaptateur HDMI Male/Femelle Pliable à 90° -Noir,0.002,1.0,500.0
1,1207,Chargeur MacBook Pro MageSafe 1 - 60W - Blanc,605,Smart Câble Displayport Mâle Vers Displayport,0.002,1.0,500.0
2,1207,Chargeur MacBook Pro MageSafe 1 - 60W - Blanc,595,Logitech Casque Logitech H340 Réducteur De Bru...,0.002,1.0,500.0
3,1207,Chargeur MacBook Pro MageSafe 1 - 60W - Blanc,605,Smart Câble Displayport Mâle Vers Displayport,0.002,1.0,500.0
4,1207,Chargeur MacBook Pro MageSafe 1 - 60W - Blanc,388,Kaspersky Plus 2024 4 Postes - 4 PC - 1 An - (...,0.002,1.0,500.0


In [104]:
#  – Build CoinAfrique rules with IDs

coin_rules_rows = []

for _, row in coin_rules.iterrows():
    antecedents = list(row["antecedents"])
    consequents = list(row["consequents"])

    for left in antecedents:
        for right in consequents:
            if left == right:
                continue

            left_id = coin_name_to_id.get(left)
            right_id = coin_name_to_id.get(right)

            if left_id is None or right_id is None:
                continue

            coin_rules_rows.append({
                "left_car_id": left_id,
                "left_car_name": left,
                "right_car_id": right_id,
                "right_car_name": right,
                "support": row["support"],
                "confidence": row["confidence"],
                "lift": row["lift"]
            })

coin_rules_id_df = pd.DataFrame(coin_rules_rows)
coin_rules_id_df = coin_rules_id_df[coin_rules_id_df["confidence"] >= 0.1]

coin_rules_id_df.head()


,left_car_id,left_car_name,right_car_id,right_car_name,support,confidence,lift
0,1699,Jetour Changan 2025,1748,Jetour Haval H2 2020,0.004,1.000000,166.666667
1,1699,Jetour Changan 2025,1256,Jetour Julion 2022,0.004,1.000000,166.666667
2,2686,Haval H9 2019,2055,Haval h6 gt 2025,0.004,1.000000,166.666667
3,1748,Jetour Haval H2 2020,1699,Jetour Changan 2025,0.004,0.666667,166.666667
4,1256,Jetour Julion 2022,1699,Jetour Changan 2025,0.004,0.666667,166.666667


In [106]:
#  – Export final rules for backend integration

jumia_rules_id_df.to_csv("jumia_rules_for_site.csv", index=False)
coin_rules_id_df.to_csv("coin_rules_for_site.csv", index=False)

print("Exported: jumia_rules_for_site.csv & coin_rules_for_site.csv")


Exported: jumia_rules_for_site.csv & coin_rules_for_site.csv


In [ ]:
### Correcting Coin Afrique ID ###


In [58]:
#La il y'a un nouveau problème, nos id dans coinAfrique ne sont pas compatibles
#Et la dans sa database, dans Coin Afrique, il a des id de 2600,4000.., moi j'en ai 1600.. ou 1200..

In [71]:
import pandas as pd
import pymysql

connection = pymysql.connect(
    host="localhost",
    user="root",
    password="placali200",   
    database="project_dm",  
    charset="utf8mb4",
    autocommit=True
)


In [73]:
coin_df = pd.read_sql("SELECT * FROM coin_afrique_cars", connection)

# 
print(coin_df.head())
print("Min ID:", coin_df["coin_afrique_id"].min(), "Max ID:", coin_df["coin_afrique_id"].max())


   coin_afrique_id       brand              model  \
0             1233  Volkswagen        passat 2013   
1             1234      Toyota         Camry 2022   
2             1235      Suzuki  grand Vitara 2023   
3             1236         Kia         Forte 2020   
4             1237  Mitsubishi        pajero 2018   

                     seller_name         location       price  \
0            Auto Boss Compagnie  Cocody, Abidjan   4800000.0   
1  georges williams athouman nze  Cocody, Abidjan  17500000.0   
2  georges williams athouman nze  Cocody, Abidjan  13900000.0   
3        DIAKITE  ( DK Company )  Cocody, Abidjan   8800000.0   
4        DIAKITE  ( DK Company )  Cocody, Abidjan   8950000.0   

                                           image_url  
0  https://images.coinafrique.com/thumb_5612140_u...  
1  https://images.coinafrique.com/thumb_5582917_u...  
2  https://images.coinafrique.com/thumb_5582929_u...  
3  https://images.coinafrique.com/thumb_5601743_u...  
4  https://imag

C:\Users\PC\AppData\Local\Temp\ipykernel_14140\564737477.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  coin_df = pd.read_sql("SELECT * FROM coin_afrique_cars", connection)


In [82]:
# D1 — Rebuild item_name from partner's database

coin_df["brand"] = coin_df["brand"].astype(str).str.strip()
coin_df["model"] = coin_df["model"].astype(str).str.strip()

coin_df["item_name"] = (coin_df["brand"] + " " + coin_df["model"]).str.strip()

coin_df[["coin_afrique_id", "item_name"]].head()


,coin_afrique_id,item_name
0,1233,Volkswagen passat 2013
1,1234,Toyota Camry 2022
2,1235,Suzuki grand Vitara 2023
3,1236,Kia Forte 2020
4,1237,Mitsubishi pajero 2018


In [85]:
#now mapping :

coin_name_to_id = dict(zip(coin_df["item_name"], coin_df["coin_afrique_id"]))

In [87]:
# D2 — Flatten CoinAfrique rules into left_name / right_name

rules_by_name = []

for _, row in coin_rules.iterrows():
    ants = list(row["antecedents"])
    cons = list(row["consequents"])
    
    for a in ants:
        for c in cons:
            if a == c:
                continue
            rules_by_name.append({
                "left_name": a,
                "right_name": c,
                "support": row["support"],
                "confidence": row["confidence"],
                "lift": row["lift"]
            })

rules_by_name_df = pd.DataFrame(rules_by_name)
rules_by_name_df.head()


,left_name,right_name,support,confidence,lift
0,KIA Forte 2017,Kia Sorento 2018,0.002,1.0,500.0
1,Honda CR-V2017,Kia Sorento 2018,0.002,1.0,500.0
2,Land Rover Range Rover 2015,Location mini bus 12 & 15 places,0.002,1.0,500.0
3,Mitsubishi RVR 2015,Hyundai Creta 2025,0.002,1.0,500.0
4,Hyundai Creta 2025,Mitsubishi RVR 2015,0.002,1.0,500.0


In [89]:
# D3 — Mapping item_name -> partner's IDs

coin_name_to_id = dict(
    zip(coin_df["item_name"], coin_df["coin_afrique_id"])
)

list(coin_name_to_id.items())[:5]


[('Volkswagen passat 2013', 1233),
 ('Toyota Camry 2022', 1665),
 ('Suzuki grand Vitara 2023', 1235),
 ('Kia Forte 2020', 1465),
 ('Mitsubishi pajero 2018', 1237)]

In [91]:
# D4 — Align rules with partner's IDs

aligned_rows = []

for _, row in rules_by_name_df.iterrows():
    left = row["left_name"]
    right = row["right_name"]
    
    left_id = coin_name_to_id.get(left)
    right_id = coin_name_to_id.get(right)
    
    if left_id is None or right_id is None:
        continue  # skip if name not found in his DB
    
    aligned_rows.append({
        "left_car_id": left_id,
        "left_car_name": left,
        "right_car_id": right_id,
        "right_car_name": right,
        "support": row["support"],
        "confidence": row["confidence"],
        "lift": row["lift"]
    })

coin_rules_partner_id_df = pd.DataFrame(aligned_rows)
coin_rules_partner_id_df.head()


,left_car_id,left_car_name,right_car_id,right_car_name,support,confidence,lift
0,2150,KIA Forte 2017,1423,Kia Sorento 2018,0.002,1.0,500.0
1,2324,Honda CR-V2017,1423,Kia Sorento 2018,0.002,1.0,500.0
2,2160,Land Rover Range Rover 2015,2515,Location mini bus 12 & 15 places,0.002,1.0,500.0
3,2264,Mitsubishi RVR 2015,2349,Hyundai Creta 2025,0.002,1.0,500.0
4,2349,Hyundai Creta 2025,2264,Mitsubishi RVR 2015,0.002,1.0,500.0


In [95]:
# D5 — Export the csv for coin afrique

coin_rules_partner_id_df.to_csv("coin_rules_for_site.csv", index=False)
print("Saved: coin_rules_for_site.csv")


Saved: coin_rules_for_site.csv
